In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import requests
import base64

In [2]:
client_id = '5fca1c7bb348462aa06215d155953a50'
client_secret = '8f4c16e4d7aa476ab71305973df02fa6'

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [15]:
user_preferences = ["5S3Gs7vbeSvGZ9t5Kp3dh9", "1mSlftOO1dlDRXAyOE0Sbd", "4dVnE4XmzKgLaKFBeRf5fi"]

In [16]:
def get_audio_features(track_uris):
    audio_features = []
    for uri in track_uris:
        track_id = uri
        track_features = sp.audio_features(track_id)
        if track_features:
            audio_features.append(track_features[0])
    return audio_features


In [17]:
user_audio_features = get_audio_features(user_preferences)

In [47]:
print(user_audio_features[1]['danceability'])

0.448


In [18]:
popular_tracks = sp.playlist_tracks('37i9dQZEVXbMDoHDwVN2tF')

In [53]:
def calculate_cosine_similarity(user_features, track_features):
    user_features = [user_features[1]['danceability'], user_features[1]['energy'], user_features[1]['valence']]
    track_features = [track_features['danceability'], track_features['energy'], track_features['valence']]
    return cosine_similarity([user_features], [track_features])[0][0]

In [54]:
similarities = []
for track in popular_tracks['items']:
    track_audio_features = sp.audio_features(track['track']['uri'])[0]
    similarity = calculate_cosine_similarity(user_audio_features, track_audio_features)
    similarities.append((track['track']['name'], track['track']['artists'][0]['name'], similarity))

In [55]:
similarities.sort(key=lambda x: x[2], reverse=True)

In [56]:
for track in similarities[:10]:
    print(f"Track: {track[0]}, Artist: {track[1]}, Similarity: {track[2]:.2f}")

Track: Lover, Artist: Taylor Swift, Similarity: 1.00
Track: As It Was, Artist: Harry Styles, Similarity: 1.00
Track: Dance The Night - From Barbie The Album, Artist: Dua Lipa, Similarity: 0.99
Track: Cruel Summer, Artist: Taylor Swift, Similarity: 0.99
Track: Que Onda, Artist: Calle 24, Similarity: 0.99
Track: un x100to, Artist: Grupo Frontera, Similarity: 0.98
Track: LADY GAGA, Artist: Peso Pluma, Similarity: 0.98
Track: Seven (feat. Latto) (Explicit Ver.), Artist: Jung Kook, Similarity: 0.98
Track: One Of Your Girls, Artist: Troye Sivan, Similarity: 0.98
Track: greedy, Artist: Tate McRae, Similarity: 0.97


In [10]:
client_credentials = f'{client_id}:{client_secret}'
base64_encoded_client_credentials = base64.b64encode(client_credentials.encode()).decode('utf-8')

In [19]:
song_name = 'despacito'

In [12]:
token_url = 'https://accounts.spotify.com/api/token'
token_data = {
    'grant_type': 'client_credentials',
}

In [13]:
token_headers = {
    'Authorization': f'Basic {base64_encoded_client_credentials}', 
}

In [14]:
token_response = requests.post(token_url, data=token_data, headers=token_headers)
token_data = token_response.json()
access_token = token_data['access_token']

In [20]:
search_url = 'https://api.spotify.com/v1/search'
search_params = {
    'q': song_name,
    'type': 'track',
}

In [21]:
search_headers = {
    'Authorization': f'Bearer {access_token}',
}

In [22]:
search_response = requests.get(search_url, params=search_params, headers=search_headers)
search_data = search_response.json()

In [23]:
if search_data['tracks']['total'] > 0:
    # Get the URI of the first result (you can loop through results if needed)
    song_uri = search_data['tracks']['items'][0]['uri']
    print(f'Song URI: {song_uri}')
else:
    print('Song not found on Spotify')


Song URI: spotify:track:6habFhsOp2NvshLv26DqMb
